In [2]:
from itertools import zip_longest

import pandas as pd

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_colwidth', None)

In [49]:
df = pd.read_csv("../dataset_checking/data/new_isaiah_dataset.csv").fillna("")
### Uncomment to exclude MN occurrences
df = df[(df.preposition_1 != "MN") & (df.preposition_2 == "")]
df.head(2)

,verb_id,lex,scroll,book,chapter,verse_num,gcons_verb,gcons_verse,sign_info,stem,tense,gcons_clause,subject,complement,cmpl_lex,cmpl_translation,dir_he,cmpl_constr,cmpl_nt,cmpl_anim,cmpl_det,cmpl_indiv,cmpl_complex,motion_type,preposition_1,preposition_2,preposition_3,preposition_4,comments
2,212418,BW>[,MT,Isaiah,1,23,JBW>,FRJK SWRRJM W XBRJ GNBJM KLW >HB CXD W RDP CLM...,,qal,impf,W RJB >LMNH L> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0,prep,prps,anim,det,prsf,simple,fictive,>L,,,,
3,1895234,BW>[,1Qisaa,Isaiah,1,23,JBW>,FRJKJ SWRRJM W XBRJ GNBJM KWLM >WHBJ CWXD RWDP...,000000 000000 0 0000 00000 00000 00000 0000 00...,qal,impf,W RJB >LMNH LW> JBW> >LJHM,RJB >LMNH,>LJHM,>L,to them,0,prep,prps,anim,det,prsf,simple,fictive,>L,,,,


In [54]:
# COMPLEMENT: OCCURRENCE COUNT

cmpl_dic = {}

for i, row in df.iterrows():
    cmpl_info = (row.lex, row.book, row.chapter, row.verse_num)
    cmpl_dic.setdefault(row.scroll, {}).setdefault(cmpl_info, []).append(row.cmpl_constr)

stats = {}
for scroll, cmpls in cmpl_dic.items():
    for cmpl_info, cmpl_constrs in cmpls.items():
        for other_scroll, other_cmpls in cmpl_dic.items():
            if scroll >= other_scroll:
                continue
            other_cmpl_constr = other_cmpls.get(cmpl_info)
            if other_cmpl_constr is None:
                continue
            same = 0
            different = 0
            for c1, c2 in zip(cmpl_constrs, other_cmpl_constr):
                if c1 == c2:
                    same += 1
                else:
                    different += 1
                    #print(scroll, other_scroll, cmpl_info, c1, c2)
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + same, stat[1] + different, stat[2] + same + different)
            
stats

{('1Qisaa', 'MT'): (165, 3, 168),
 ('1Q8', 'MT'): (65, 0, 65),
 ('1Q8', '1Qisaa'): (62, 1, 63)}

In [50]:
#df[(df.chapter == 52) & (df.verse_num == 11)]

In [55]:
# COMPLEMENT: OCCURRENCE COUNT AND PERCENTAGES, TABLE

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": len(df[df.scroll == scrolls[0]]),
        "total_occ_scroll2": len(df[df.scroll == scrolls[1]]),
        "total_com_occ": stat[2],
        "total_agreement": stat[0],
        "total_disagreement": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_agreement,total_disagreement,agreement_percentage
0,1Qisaa,MT,174,175,168,165,3,98.214286
1,1Q8,MT,65,175,65,65,0,100.000000
2,1Q8,1Qisaa,65,174,63,62,1,98.412698


In [47]:
# COMPLEMENT: DISAGREEMENT CASES
cmpl_dic = {}

# Constructing the dictionary from the dataframe
for i, row in df.iterrows():
    cmpl_info = (row.lex, row.book, row.chapter, row.verse_num)
    cmpl_dic.setdefault(row.scroll, {}).setdefault(cmpl_info, []).append(row.cmpl_constr)

results = []

# Analyzing and comparing complements
for scroll, cmpls in cmpl_dic.items():
    for cmpl_info, cmpl_constrs in cmpls.items():
        for other_scroll, other_cmpls in cmpl_dic.items():
            if scroll >= other_scroll:
                continue
            other_cmpl_constr = other_cmpls.get(cmpl_info)
            if other_cmpl_constr is None:
                continue
            
            for c1, c2 in zip(cmpl_constrs, other_cmpl_constr):
                if c1 != c2:  # Only storing results when they are different
                    result = {
                        'scroll': scroll,
                        'other_scroll': other_scroll,
                        'lex': cmpl_info[0],
                        'book': cmpl_info[1],
                        'chapter': cmpl_info[2],
                        'verse_num': cmpl_info[3],
                        'complement_1': c1,
                        'complement_2': c2
                    }
                    results.append(result)

# Create DataFrame
df_results = pd.DataFrame(results)

# If you want to see the DataFrame printed out
df_results
#stats

,scroll,other_scroll,lex,book,chapter,verse_num,complement_1,complement_2
0,1Qisaa,MT,BW>[,Isaiah,10,28,prep + dir-he,prep
1,1Qisaa,MT,<BR[,Isaiah,10,29,prep,vc
2,1Qisaa,MT,<LH[,Isaiah,60,7,prep,vc
3,1Q8,1Qisaa,<LH[,Isaiah,60,7,vc,prep


In [46]:
# PREPOSITIONS: COUNT

prep_dic = {}

for i, row in df[df.cmpl_constr == "prep"].iterrows():
    prep_info = (row.lex, row.book, row.chapter, row.verse_num)
    preps = []
    for i in range(1, 5):
        prep = row[f"preposition_{i}"]
        if prep:
            preps.append(prep)
    prep_dic.setdefault(row.scroll, {}).setdefault(prep_info, []).append(preps)

stats = {}
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll >= other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            same = 0
            different = 0
            for c1, c2 in zip(preps, other_preps):
                if c1 == c2:
                    same += 1
                else:
                    different += 1
                    #print(scroll, other_scroll, prep_info, c1, c2)
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + same, stat[1] + different, stat[2] + same + different)

stats

{('1Qisaa', 'MT'): (125, 9, 134),
 ('1Q8', 'MT'): (49, 1, 50),
 ('1Q8', '1Qisaa'): (45, 4, 49)}

In [41]:
# PREPOSITIONS. DISAGREEMENT CASES BETWEEN THE THREE MANUSCRIPTS
prep_dic = {}

# Constructing the dictionary from the dataframe where 'cmpl_constr' is 'prep'
for i, row in df[df.cmpl_constr == "prep"].iterrows():
    prep_info = (row.lex, row.book, row.chapter, row.verse_num)
    preps = []
    for j in range(1, 5):
        prep = row[f"preposition_{j}"]
        if prep:
            preps.append(prep)
    prep_dic.setdefault(row.scroll, {}).setdefault(prep_info, []).append(preps)

results = []

# Analyzing and comparing prepositions
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll >= other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            
            for c1, c2 in zip(preps, other_preps):
                if c1 != c2:  # Only storing results when they are different
                    result = {
                        'scroll': scroll,
                        'other_scroll': other_scroll,
                        'lex': prep_info[0],
                        'book': prep_info[1],
                        'chapter': prep_info[2],
                        'verse_num': prep_info[3],
                        'preposition_scroll_1': ", ".join(c1),
                        'preposition_scroll_2': ", ".join(c2)
                    }
                    results.append(result)

# Create DataFrame
df_results = pd.DataFrame(results)

# If you want to see the DataFrame printed out
df_results

,scroll,other_scroll,lex,book,chapter,verse_num,preposition_scroll_1,preposition_scroll_2
0,1Qisaa,MT,NHR[,Isaiah,2,2,<L,>L
1,1Qisaa,MT,BW>[,Isaiah,16,12,<L,>L
2,1Qisaa,MT,HLK[,Isaiah,18,2,L,>L
3,1Qisaa,MT,CWB[,Isaiah,37,7,L,>L
4,1Qisaa,MT,GLH[,Isaiah,53,1,>L,<L
5,1Qisaa,MT,<BR[,Isaiah,54,9,<L,<L
6,1Qisaa,MT,BW>[,Isaiah,59,20,>L,L
7,1Qisaa,MT,BW>[,Isaiah,60,11,>L,>L
8,1Qisaa,MT,BW>[,Isaiah,66,20,>L,<L
9,1Q8,1Qisaa,BW>[,Isaiah,16,12,>L,<L


In [26]:
# PREPOSITIONS: MANUSCRIPT AGREEMENT

def total_occ(scroll):
    scroll_preps = prep_dic[scroll]
    total = 0
    for preps in scroll_preps.values():
        total += len(preps)
    return total

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": total_occ(scrolls[0]),
        "total_occ_scroll2": total_occ(scrolls[1]),
        "total_com_occ": stat[2],
        "total_agreement": stat[0],
        "total_disagreement": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_agreement,total_disagreement,agreement_percentage
0,1Qisaa,MT,142,139,134,125,9,93.283582
1,1Q8,MT,50,139,50,49,1,98.000000
2,1Q8,1Qisaa,50,142,49,45,4,91.836735


In [43]:
# PREPOSITION MISSING

stats = {}
for scroll, scroll_preps in prep_dic.items():
    for prep_info, preps in scroll_preps.items():
        for other_scroll, other_scroll_preps in prep_dic.items():
            if scroll == other_scroll:
                continue
            other_preps = other_scroll_preps.get(prep_info)
            if other_preps is None:
                continue
            non_missing = 0
            missing = 0
            for c1, c2 in zip_longest(preps, other_preps):
                if c2 is None:
                    missing += 1
                    print(scroll, other_scroll, prep_info, c1, c2)
                else:
                    non_missing += 1
            stat = stats.get((scroll, other_scroll), [0, 0, 0])
            stats[(scroll, other_scroll)] = (stat[0] + non_missing, stat[1] + missing, stat[2] + missing + non_missing)
            
stats

1Qisaa MT ('BW>[', 'Isaiah', 16, 12) ['>L '] None
1Qisaa 1Q8 ('BW>[', 'Isaiah', 16, 12) ['>L '] None


{('MT', '1Qisaa'): (135, 0, 135),
 ('MT', '1Q8'): (50, 0, 50),
 ('1Qisaa', 'MT'): (134, 1, 135),
 ('1Qisaa', '1Q8'): (49, 1, 50),
 ('1Q8', 'MT'): (50, 0, 50),
 ('1Q8', '1Qisaa'): (50, 0, 50)}

In [44]:
# PREPOSITION MISSING 

pd.DataFrame([{
        "scroll1": scrolls[0], 
        "scroll2": scrolls[1], 
        "total_occ_scroll1": total_occ(scrolls[0]),
        "total_occ_scroll2": total_occ(scrolls[1]),
        "total_com_occ": stat[2],
        "total_non_missing": stat[0],
        "total_missing_from_scroll2": stat[1],
        "agreement_percentage": stat[0] / stat[2] * 100,
    }
    for scrolls, stat in stats.items()
])

,scroll1,scroll2,total_occ_scroll1,total_occ_scroll2,total_com_occ,total_non_missing,total_missing_from_scroll2,agreement_percentage
0,MT,1Qisaa,139,142,135,135,0,100.000000
1,MT,1Q8,139,50,50,50,0,100.000000
2,1Qisaa,MT,142,139,135,134,1,99.259259
3,1Qisaa,1Q8,142,50,50,49,1,98.000000
4,1Q8,MT,50,139,50,50,0,100.000000
5,1Q8,1Qisaa,50,142,50,50,0,100.000000
